In [6]:
import os, sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from utils.helpers import launch_env, wrap_env, view_results_ipython, change_exercise, seedall, force_done, evaluate_policy
from utils.helpers import SteeringToWheelVelWrapper, ResizeWrapper, ImgWrapper

import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

INFO:aido-protocols:aido-protocols 5.0.5
22:22:41|zn|__init__.py:6|<module>(): zn 2.0.3
22:22:42|zj|__init__.py:5|<module>(): zj 2.0.4
22:22:42|gym-duckietown|__init__.py:10|<module>(): gym-duckietown 2019.0.0

22:22:42|gym-duckietown|__init__.py:24|reg_map_env(): Registering gym environment id: Duckietown-loop_pedestrians-v0
22:22:42|gym-duckietown|__init__.py:24|reg_map_env(): Registering gym environment id: Duckietown-loop_dyn_duckiebots-v0
22:22:42|gym-duckietown|__init__.py:24|reg_map_env(): Registering gym environment id: Duckietown-loop_empty-v0
22:22:42|gym-duckietown|__init__.py:24|reg_map_env(): Registering gym environment id: Duckietown-small_loop_cw-v0
22:22:42|gym-duckietown|__init__.py:24|reg_map_env(): Registering gym environment id: Duckietown-small_loop-v0
22:22:42|gym-duckietown|__init__.py:24|reg_map_env(): Registering gym environment id: Duckietown-4way-v0
22:22:42|gym-duckietown|__init__.py:24|reg_map_env(): Registering gym environment id: Duckietown-udem1-v0
22:22

# Reinforcement Learning Basics

Reinforcement Learning, as we saw in lecture, is the idea of learning a _policy_ in order to maximize future (potentially discounted) rewards. Our policy, similar to the imitation learning network, maps raw image observations to wheel velocities, and at every timestep, receives a _reward_ from the environment. 

Rewards can be sparse (`1` if goal or task is completed, `0` otherwise) or dense; in general, dense rewards make it easier to learn policies, but as we'll see later in this exercise, defining the correct dense reward is an engineering challenge on its own.

Today's reinforcement learning algorithms are often a mix between _value-based_ and _policy-gradient_ algorithms, instances of what is called an _actor-critic_ formulation. Actor-critic methods have had a lot of research done on them in recent years (especially within in the deep reinforcement learning era), and later in this exercise, we shall also rediscover the formulation's original problems and different methods currently used to stabilize learning.

We begin by defining two networks, an `Actor` and `Critic`; in this exercise, we'll be using a deep RL algorithm titled _Deep Deterministic Policy Gradients_. 

In [7]:
class Actor(nn.Module):
    def __init__(self, action_dim, max_action):
        super(Actor, self).__init__()

        # TODO: You'll need to change this!
        flat_size = 31968

        self.relu = nn.ReLU()
        self.tanh = nn.Tanh()

        self.conv1 = nn.Conv2d(3, 32, 8, stride=2)
        self.conv2 = nn.Conv2d(32, 32, 4, stride=2)

        self.bn1 = nn.BatchNorm2d(32)
        self.bn2 = nn.BatchNorm2d(32)

        self.dropout = nn.Dropout(.1)

        self.lin1 = nn.Linear(flat_size, 100)
        self.lin2 = nn.Linear(100, action_dim)

        self.max_action = max_action

    def forward(self, x):
        x = self.bn1(self.relu(self.conv1(x)))
        x = self.bn2(self.relu(self.conv2(x)))
        x = x.view(x.size(0), -1)  # flatten
        x = self.dropout(x)
        x = self.relu(self.lin1(x))

        x = self.lin2(x)
        x = self.max_action * self.tanh(x)
        
        return x
    
class Critic(nn.Module):
    def __init__(self, action_dim, max_action):
        super(Critic, self).__init__()

        # TODO: You'll need to change this!
        flat_size = 31968

        self.relu = nn.ReLU()
        self.tanh = nn.Tanh()

        self.conv1 = nn.Conv2d(3, 32, 8, stride=2)
        self.conv2 = nn.Conv2d(32, 32, 4, stride=2)

        self.bn1 = nn.BatchNorm2d(32)
        self.bn2 = nn.BatchNorm2d(32)

        self.dropout = nn.Dropout(.1)

        self.lin1 = nn.Linear(flat_size + action_dim, 100)
        self.lin2 = nn.Linear(100, action_dim)

        self.max_action = max_action

    def forward(self, obs, action):
        x = self.bn1(self.relu(self.conv1(obs)))
        x = self.bn2(self.relu(self.conv2(x)))
        x = x.view(x.size(0), -1)  # flatten
        x = self.dropout(x)
        x = torch.cat([x, action], 1)
        x = self.relu(self.lin1(x))

        x = self.lin2(x)
        x = self.max_action * self.tanh(x)
        
        return x

## Reward Engineering

In this part of the exercise, we will experiment with the reward formulation. Given the same model, we'll see how the effect of various reward functions changes the final policy trained. 

In the section below, we'll take a look at the reward function implemented in `gym-duckietown` with a slightly modified training loop. Traditionally, we `reset()` the environment to start an episode, and then `step()` the environment forward for a set amount of time, executing a new action. If this sounds a bit odd, especially for roboticists, you're right - in real robotics, most code runs asynchronously. As a result, although `gym-duckietown` runs locally by stopping the environment, the `AIDO` submissions will run asynchronously, executing the same action until a new one is received.

In [63]:
def updated_reward(env):
    # Compute the collision avoidance penalty
    pos, angle, speed = env.cur_pos, env.cur_angle, env.speed
    col_penalty = env.proximity_penalty2(pos, angle)
    print("col_penalty: {}".format(col_penalty))

    # Get the position relative to the right lane tangent
    try:
        lp = env.get_lane_pos2(pos, angle)
    except NotInLane:
        reward = 100 * col_penalty
    else:
        print("dot_dir: {}".format(lp.dot_dir))
        print("dist: {}".format(np.abs(lp.dist)))
        
        # Compute the reward
        reward = (
                +0 * speed * lp.dot_dir +
                -10 * np.abs(lp.dist) +
                +0 * col_penalty
        )
    return reward

In [8]:
nepisodes = 10

In [64]:
local_env = launch_env()
local_env = wrap_env(local_env)
local_env = ResizeWrapper(local_env)
local_env = ImgWrapper(local_env)

for _ in range(nepisodes):
    done = False
    obs = local_env.reset()
    
    while not done:
        obs, r, done, info = local_env.step(np.random.random(2))
        new_r = updated_reward(local_env)
        print(r, new_r)
 

14:00:54|gym-duckietown|graphics.py:121|create_frame_buffers(): Falling back to non-multisampled frame buffer
14:00:54|gym-duckietown|graphics.py:121|create_frame_buffers(): Falling back to non-multisampled frame buffer
14:00:54|gym-duckietown|simulator.py:550|_load_map(): loading map file "/duckietown/simulation/gym_duckietown/maps/loop_empty.yaml"
14:00:54|gym-duckietown|simulator.py:1107|_drivable_pos(): [4.13719561 0.         1.88880899] corresponds to tile at (7, 3) which is not drivable: {'coords': (7, 3), 'kind': 'floor', 'angle': 0, 'drivable': False, 'texture': <simulation.gym_duckietown.graphics.Texture object at 0x7fb0c9fdaa20>, 'color': array([1, 1, 1])}
14:00:54|gym-duckietown|simulator.py:1217|_valid_pose(): Invalid pose. Collision free: True On drivable area: False
14:00:54|gym-duckietown|simulator.py:1218|_valid_pose(): safety_factor: 1.3
14:00:54|gym-duckietown|simulator.py:1219|_valid_pose(): pos: [4.0873995 0.        1.9726338]
14:00:54|gym-duckietown|simulator.py:12

14:00:54|gym-duckietown|simulator.py:1217|_valid_pose(): Invalid pose. Collision free: True On drivable area: False
14:00:54|gym-duckietown|simulator.py:1218|_valid_pose(): safety_factor: 1.3
14:00:54|gym-duckietown|simulator.py:1219|_valid_pose(): pos: [3.57292165 0.         1.88314807]
14:00:54|gym-duckietown|simulator.py:1220|_valid_pose(): l_pos: [3.48172154 0.         1.84866924]
14:00:55|gym-duckietown|simulator.py:1221|_valid_pose(): r_pos: [3.66412176 0.         1.91762689]
14:00:55|gym-duckietown|simulator.py:1222|_valid_pose(): f_pos: [3.61429624 0.         1.77370793]
14:00:55|gym-duckietown|simulator.py:1107|_drivable_pos(): [3.50690495 0.         2.23654429] corresponds to tile at (5, 3) which is not drivable: {'coords': (5, 3), 'kind': 'floor', 'angle': 0, 'drivable': False, 'texture': <simulation.gym_duckietown.graphics.Texture object at 0x7fb0c9fdaa20>, 'color': array([1, 1, 1])}
14:00:55|gym-duckietown|simulator.py:1217|_valid_pose(): Invalid pose. Collision free: True

14:00:55|gym-duckietown|simulator.py:1222|_valid_pose(): f_pos: [3.50609039 0.         2.05544313]
14:00:55|gym-duckietown|simulator.py:1107|_drivable_pos(): [4.12972415 0.         2.27350789] corresponds to tile at (7, 3) which is not drivable: {'coords': (7, 3), 'kind': 'floor', 'angle': 0, 'drivable': False, 'texture': <simulation.gym_duckietown.graphics.Texture object at 0x7fb0c9fdaa20>, 'color': array([1, 1, 1])}
14:00:55|gym-duckietown|simulator.py:1217|_valid_pose(): Invalid pose. Collision free: True On drivable area: False
14:00:55|gym-duckietown|simulator.py:1218|_valid_pose(): safety_factor: 1.3
14:00:55|gym-duckietown|simulator.py:1219|_valid_pose(): pos: [4.022444   0.         2.32019802]
14:00:55|gym-duckietown|simulator.py:1220|_valid_pose(): l_pos: [3.98353555 0.         2.23079789]
14:00:55|gym-duckietown|simulator.py:1221|_valid_pose(): r_pos: [4.06135244 0.         2.40959815]
14:00:55|gym-duckietown|simulator.py:1222|_valid_pose(): f_pos: [4.12972415 0.         2.27

14:01:01|gym-duckietown|simulator.py:1107|_drivable_pos(): [2.28873639 0.         2.68406941] corresponds to tile at (3, 4) which is not drivable: {'coords': (3, 4), 'kind': 'floor', 'angle': 0, 'drivable': False, 'texture': <simulation.gym_duckietown.graphics.Texture object at 0x7fb0c9fdaa20>, 'color': array([1, 1, 1])}
14:01:01|gym-duckietown|simulator.py:1217|_valid_pose(): Invalid pose. Collision free: True On drivable area: False
14:01:01|gym-duckietown|simulator.py:1218|_valid_pose(): safety_factor: 1.3
14:01:01|gym-duckietown|simulator.py:1219|_valid_pose(): pos: [2.38071712 0.         2.71640816]
14:01:01|gym-duckietown|simulator.py:1220|_valid_pose(): l_pos: [2.28873639 0.         2.68406941]
14:01:01|gym-duckietown|simulator.py:1221|_valid_pose(): r_pos: [2.47269785 0.         2.74874692]
14:01:01|gym-duckietown|simulator.py:1222|_valid_pose(): f_pos: [2.41952363 0.         2.60603129]
14:01:01|gym-duckietown|simulator.py:531|reset(): Starting at [2.64153703 0.         2.8846

col_penalty: 0.0
dot_dir: 0.9999505871536081
dist: 0.13100253097773257
-0.11008460519299601 -1.3100253097773256
col_penalty: 0.0
dot_dir: 0.9999505871536081
dist: 0.13100253097773257
-0.11008460519299601 -1.3100253097773256
col_penalty: 0.0
dot_dir: 0.9999505871536081
dist: 0.13100253097773257
-0.11008460519299601 -1.3100253097773256
col_penalty: 0.0
dot_dir: 0.9999505871536081
dist: 0.13100253097773257
-0.11008460519299601 -1.3100253097773256


...

col_penalty: 0.0
dot_dir: 0.6082490913857475
dist: 0.3727960211801447
-2.99806130213855 -3.727960211801447
col_penalty: 0.0
dot_dir: 0.615657151943985
dist: 0.37927313467079005
-3.0539427643751185 -3.7927313467079005
col_penalty: 0.0
dot_dir: 0.624270382386249
dist: 0.38515066572863266
-3.1023821984228275 -3.8515066572863264
col_penalty: 0.0
dot_dir: 0.6201125215114895
dist: 0.39132682037415056
-3.1691331779277183 -3.913268203741506
col_penalty: 0.0
dot_dir: 0.6154556537131147
dist: 0.3972883070912122
-3.2343362864563843 -3.972883070912122
col_penalty: 0.0
dot_dir: 0.6013535197474422
dist: 0.40325364169841915
-3.310912193287261 -4.0325364169841915
col_penalty: 0.0


14:01:15|gym-duckietown|simulator.py:1435|_render_img(): Pos: [3.910208945147194, 0, 2.790858939099191] angle -1.4699857681112476
14:01:15|gym-duckietown|simulator.py:1435|_render_img(): Pos: [3.9110441095697843, 0, 2.799175420635332] angle -1.4714323734444996
14:01:15|gym-duckietown|simulator.py:1435|_render_img(): Pos: [3.911880548661359, 0, 2.8079216741751987] angle -1.4794722317537616
14:01:15|gym-duckietown|simulator.py:1435|_render_img(): Pos: [3.912766005816964, 0, 2.8180744123358688] angle -1.488133397126533
14:01:15|gym-duckietown|simulator.py:1435|_render_img(): Pos: [3.913530495953758, 0, 2.827687701487367] angle -1.4947446609019548
14:01:16|gym-duckietown|simulator.py:1435|_render_img(): Pos: [3.9143243607157574, 0, 2.8381821202283053] angle -1.495842835430861
14:01:16|gym-duckietown|simulator.py:1435|_render_img(): Pos: [3.915040479073269, 0, 2.8477138283587378] angle -1.4957713446955465
14:01:16|gym-duckietown|simulator.py:1435|_render_img(): Pos: [3.915720228645665, 0, 2

dot_dir: 0.5887132516853104
dist: 0.40957812219872713
-3.3893253199648994 -4.095781221987272
col_penalty: 0.0
dot_dir: 0.5796333794558545
dist: 0.4168366574092425
-3.4728065187454 -4.168366574092425
col_penalty: 0.0
dot_dir: 0.5744657374128989
dist: 0.42366491766229875
-3.5472902917275086 -4.236649176622987
col_penalty: 0.0
dot_dir: 0.5746906400667764
dist: 0.4308631271623007
-3.619002503542875 -4.308631271623007
col_penalty: 0.0
dot_dir: 0.5690808848760996
dist: 0.43922153041884576
-3.709318242337138 -4.392215304188458
col_penalty: 0.0
dot_dir: 0.5681583909679456
dist: 0.44714653675980337
-3.7896752984364994 -4.471465367598034
col_penalty: 0.0
dot_dir: 0.5626966578152792
dist: 0.45582911103714174
-3.8830551209930824 -4.558291110371417
col_penalty: 0.0
dot_dir: 0.5562498144750248
dist: 0.4637627047505124
-3.970127270135094 -4.637627047505124


14:01:16|gym-duckietown|simulator.py:1435|_render_img(): Pos: [3.9163349629977677, 0, 2.866583356831371] angle -1.5102123054942878
14:01:16|gym-duckietown|simulator.py:1107|_drivable_pos(): [3.92033106 0.         2.93246227] corresponds to tile at (6, 5) which is not drivable: {'coords': (6, 5), 'kind': 'floor', 'angle': 0, 'drivable': False, 'texture': <simulation.gym_duckietown.graphics.Texture object at 0x7fb0c9fdaa20>, 'color': array([1, 1, 1])}
14:01:16|gym-duckietown|simulator.py:1217|_valid_pose(): Invalid pose. Collision free: True On drivable area: False
14:01:16|gym-duckietown|simulator.py:1218|_valid_pose(): safety_factor: 1.0
14:01:16|gym-duckietown|simulator.py:1219|_valid_pose(): pos: [3.91488184 0.         2.84262739]
14:01:16|gym-duckietown|simulator.py:1220|_valid_pose(): l_pos: [3.98974424 0.         2.83808637]
14:01:16|gym-duckietown|simulator.py:1221|_valid_pose(): r_pos: [3.84001943 0.         2.84716841]
14:01:16|gym-duckietown|simulator.py:1222|_valid_pose(): f_

col_penalty: 0.0
dot_dir: 0.5482915750373857
dist: 0.47162593794431795
-4.058309489398317 -4.71625937944318
col_penalty: 0.0
dot_dir: 0.5490915257605882
dist: 0.4795374183851991
-1000 -4.795374183851991


In [65]:
view_results_ipython(local_env)

**Question 0: After understanding the above computed reward, experiment with the constants for each component. What type of behavior does the above reward function penalize? Is this good or bad in context of autonomous driving? Name some other issues that can arise with single-objective optimization. In addition, give three sets of constants and explain qualitatively what types of behavior each penalizes or rewards (note, you may want to use a different action policy than random)**. Place the answers to the above in `reinforcement-learning-answers.txt`




# The Reinforcement Learning Learning Code

Below we'll see a relatively naive implementation of the actor-critic training loop, which proceeds as follows: the critic is tasked with a supervised learning problem of fitting rewards acquired by the agent. Then, the policy, using policy gradients, maximizes the return according to the critic's estimate, rather than using Monte-Carlo updates.

Below, we see an implementation of `DDPGAgent`, a class which handles the networks and training loop. 

In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class DDPGAgent(object):
    def __init__(self, state_dim, action_dim, max_action=1.0):
        super(DDPGAgent, self).__init__()

        self.actor = Actor(action_dim, max_action).to(device)
        self.actor_optimizer = torch.optim.Adam(self.actor.parameters(), lr=5e-2)
        
        self.critic = Critic(action_dim, max_action).to(device)
        self.critic_optimizer = torch.optim.Adam(self.critic.parameters(), lr=5e-2)
        
        self.flat = False

    def predict(self, state):
        assert state.shape[0] == 3
        state = torch.FloatTensor(np.expand_dims(state, axis=0)).to(device)
        return self.actor(state).cpu().data.numpy().flatten()

    def train(self, replay_buffer, iterations, batch_size=64, discount=0.99):
        for it in range(iterations):

            # Sample replay buffer
            sample = replay_buffer.sample(batch_size, flat=self.flat)
            state = torch.FloatTensor(sample["state"]).to(device)
            action = torch.FloatTensor(sample["action"]).to(device)
            next_state = torch.FloatTensor(sample["next_state"]).to(device)
            done = torch.FloatTensor(1 - sample["done"]).to(device)
            reward = torch.FloatTensor(sample["reward"]).to(device)

            # Compute the target Q value
            target_Q = self.critic(next_state, self.actor(next_state))
            
            # TODO: - no detach is a subtle, but important bug!
            target_Q = reward + (done * discount * target_Q)
            target_Q = target_Q.detach()

            # Get current Q estimate
            current_Q = self.critic(state, action)

            # Compute critic loss
            critic_loss = F.mse_loss(current_Q, target_Q)

            # Optimize the critic
            self.critic_optimizer.zero_grad()
            critic_loss.backward()
            self.critic_optimizer.step()

            # Compute actor loss
            actor_loss = -self.critic(state, self.actor(state)).mean()

            # Optimize the actor
            self.actor_optimizer.zero_grad()
            actor_loss.backward()
            self.actor_optimizer.step()
            
    def save(self, filename, directory):
        torch.save(self.actor.state_dict(), '{}/{}_actor.pth'.format(directory, filename))
        torch.save(self.critic.state_dict(), '{}/{}_critic.pth'.format(directory, filename))

    def load(self, filename, directory):
        self.actor.load_state_dict(torch.load('{}/{}_actor.pth'.format(directory, filename), map_location=device))
        self.critic.load_state_dict(torch.load('{}/{}_critic.pth'.format(directory, filename), map_location=device))


You'll notice that the training loop needs a `replay_buffer` object. In value-based and actor-critic methods in deep reinforcement learning, the use of a replay buffer is crucial. In the following sections, you'll explore why this is the case, and some other stabilization techniques that are needed in order to get the above code to work. Below, you can find an implementation of the replay buffer, as well the training loop that we use to train DDPG.

In [13]:
# Simple replay buffer
class ReplayBuffer(object):
    def __init__(self, max_size=10e3):
        self.storage = []
        self.max_size = max_size

    # Expects tuples of (state, next_state, action, reward, done)
    def add(self, state, next_state, action, reward, done):
        if len(self.storage) < self.max_size:
            self.storage.append((state, next_state, action, reward, done))
        else:
            # Remove random element in the memory beforea adding a new one
            self.storage.pop(random.randrange(len(self.storage)))
            self.storage.append((state, next_state, action, reward, done))


    def sample(self, batch_size=100, flat=True):
        ind = np.random.randint(0, len(self.storage), size=batch_size)
        states, next_states, actions, rewards, dones = [], [], [], [], []

        for i in ind:
            state, next_state, action, reward, done = self.storage[i]

            if flat:
                states.append(np.array(state, copy=False).flatten())
                next_states.append(np.array(next_state, copy=False).flatten())
            else:
                states.append(np.array(state, copy=False))
                next_states.append(np.array(next_state, copy=False))
            actions.append(np.array(action, copy=False))
            rewards.append(np.array(reward, copy=False))
            dones.append(np.array(done, copy=False))

        # state_sample, action_sample, next_state_sample, reward_sample, done_sample
        return {
            "state": np.stack(states),
            "next_state": np.stack(next_states),
            "action": np.stack(actions),
            "reward": np.stack(rewards).reshape(-1,1),
            "done": np.stack(dones).reshape(-1,1)
        }

In [14]:
seed_ = 123
env_timesteps = 1e3
max_timesteps = 1e5
batch_size = 64
discount = 0.99
eval_freq = 5e3
file_name = 'dt-class-rl'
start_timesteps = 1e4
expl_noise = 0.1

In [16]:
import os
import time

local_env = launch_env()
#local_env = wrap_env(local_env)
local_env = ResizeWrapper(local_env)
local_env = ImgWrapper(local_env)

if not os.path.exists("./pytorch_models"):
    os.makedirs("./pytorch_models")

# Set seeds
seedall(seed_)

state_dim = local_env.observation_space.shape
action_dim = local_env.action_space.shape[0]
max_action = float(local_env.action_space.high[0])

# Initialize policy
policy = DDPGAgent(state_dim, action_dim, max_action)

replay_buffer = ReplayBuffer()

# Evaluate untrained policy
evaluations= [evaluate_policy(local_env, policy)]

total_timesteps = 0
timesteps_since_eval = 0
episode_num = 0
done = True
episode_reward = None
env_counter = 0

start = time.time()
while total_timesteps < max_timesteps:
    if done:
        print("EPISODE DONE IN {}".format(time.time() - start))
        if total_timesteps != 0:
            print(("Total T: %d Episode Num: %d Episode T: %d Reward: %f") % (
                total_timesteps, episode_num, episode_timesteps, episode_reward))
            policy.train(replay_buffer, episode_timesteps, batch_size, discount)

        # Evaluate episode
        if timesteps_since_eval >= eval_freq:
            timesteps_since_eval %= eval_freq
            evaluations.append(evaluate_policy(local_env, policy))

            policy.save(file_name, directory="./pytorch_models")
            np.savez("./pytorch_models/{}.npz".format(file_name),evaluations)

        # Reset environment
        env_counter += 1
        obs = local_env.reset()
        done = False
        episode_reward = 0
        episode_timesteps = 0
        episode_num += 1

    # Select action randomly or according to policy
    if total_timesteps < start_timesteps:
        action = local_env.action_space.sample()
    else:
        action = policy.predict(np.array(obs))
        if expl_noise != 0:
            action = (action + np.random.normal(
                0,
                expl_noise,
                size=local_env.action_space.shape[0])
            ).clip(-1, +1)

    # Perform action
    new_obs, reward, done, _ = local_env.step(action)

    if episode_timesteps >= env_timesteps:
        done = True

    done_bool = 0 if episode_timesteps + 1 == env_timesteps else float(done)
    episode_reward += reward

    # Store data in replay buffer
    replay_buffer.add(obs, new_obs, action, reward, done_bool)

    obs = new_obs

    episode_timesteps += 1
    total_timesteps += 1
    timesteps_since_eval += 1

# Final evaluation
evaluations.append(evaluate_policy(local_env, policy))

if args.save_models:
    policy.save(file_name, directory="./pytorch_models")
np.savez("./pytorch_models/{}.npz".format(file_name),evaluations)

22:42:18|gym-duckietown|graphics.py:121|create_frame_buffers(): Falling back to non-multisampled frame buffer
22:42:18|gym-duckietown|graphics.py:121|create_frame_buffers(): Falling back to non-multisampled frame buffer
22:42:18|gym-duckietown|simulator.py:550|_load_map(): loading map file "/duckietown/simulation/gym_duckietown/maps/loop_empty.yaml"
22:42:18|gym-duckietown|simulator.py:1107|_drivable_pos(): [4.13719561 0.         1.88880899] corresponds to tile at (7, 3) which is not drivable: {'coords': (7, 3), 'kind': 'floor', 'angle': 0, 'drivable': False, 'texture': <simulation.gym_duckietown.graphics.Texture object at 0x7fbd8d100860>, 'color': array([1, 1, 1])}
22:42:18|gym-duckietown|simulator.py:1217|_valid_pose(): Invalid pose. Collision free: True On drivable area: False
22:42:18|gym-duckietown|simulator.py:1218|_valid_pose(): safety_factor: 1.3
22:42:18|gym-duckietown|simulator.py:1219|_valid_pose(): pos: [4.0873995 0.        1.9726338]
22:42:18|gym-duckietown|simulator.py:12

22:42:18|gym-duckietown|simulator.py:1217|_valid_pose(): Invalid pose. Collision free: True On drivable area: False
22:42:18|gym-duckietown|simulator.py:1218|_valid_pose(): safety_factor: 1.3
22:42:18|gym-duckietown|simulator.py:1219|_valid_pose(): pos: [3.57292165 0.         1.88314807]
22:42:18|gym-duckietown|simulator.py:1220|_valid_pose(): l_pos: [3.48172154 0.         1.84866924]
22:42:18|gym-duckietown|simulator.py:1221|_valid_pose(): r_pos: [3.66412176 0.         1.91762689]
22:42:18|gym-duckietown|simulator.py:1222|_valid_pose(): f_pos: [3.61429624 0.         1.77370793]
22:42:18|gym-duckietown|simulator.py:1107|_drivable_pos(): [3.50690495 0.         2.23654429] corresponds to tile at (5, 3) which is not drivable: {'coords': (5, 3), 'kind': 'floor', 'angle': 0, 'drivable': False, 'texture': <simulation.gym_duckietown.graphics.Texture object at 0x7fbd8d100860>, 'color': array([1, 1, 1])}
22:42:18|gym-duckietown|simulator.py:1217|_valid_pose(): Invalid pose. Collision free: True

22:42:18|gym-duckietown|simulator.py:1222|_valid_pose(): f_pos: [3.50609039 0.         2.05544313]
22:42:18|gym-duckietown|simulator.py:1107|_drivable_pos(): [4.12972415 0.         2.27350789] corresponds to tile at (7, 3) which is not drivable: {'coords': (7, 3), 'kind': 'floor', 'angle': 0, 'drivable': False, 'texture': <simulation.gym_duckietown.graphics.Texture object at 0x7fbd8d100860>, 'color': array([1, 1, 1])}
22:42:18|gym-duckietown|simulator.py:1217|_valid_pose(): Invalid pose. Collision free: True On drivable area: False
22:42:18|gym-duckietown|simulator.py:1218|_valid_pose(): safety_factor: 1.3
22:42:18|gym-duckietown|simulator.py:1219|_valid_pose(): pos: [4.022444   0.         2.32019802]
22:42:18|gym-duckietown|simulator.py:1220|_valid_pose(): l_pos: [3.98353555 0.         2.23079789]
22:42:18|gym-duckietown|simulator.py:1221|_valid_pose(): r_pos: [4.06135244 0.         2.40959815]
22:42:18|gym-duckietown|simulator.py:1222|_valid_pose(): f_pos: [4.12972415 0.         2.27

22:42:25|gym-duckietown|simulator.py:1217|_valid_pose(): Invalid pose. Collision free: True On drivable area: False
22:42:25|gym-duckietown|simulator.py:1218|_valid_pose(): safety_factor: 1.3
22:42:25|gym-duckietown|simulator.py:1219|_valid_pose(): pos: [2.38071712 0.         2.71640816]
22:42:25|gym-duckietown|simulator.py:1220|_valid_pose(): l_pos: [2.28873639 0.         2.68406941]
22:42:25|gym-duckietown|simulator.py:1221|_valid_pose(): r_pos: [2.47269785 0.         2.74874692]
22:42:25|gym-duckietown|simulator.py:1222|_valid_pose(): f_pos: [2.41952363 0.         2.60603129]
22:42:25|gym-duckietown|simulator.py:531|reset(): Starting at [2.64153703 0.         2.88468928] 4.836049502872963
22:42:25|gym-duckietown|simulator.py:1435|_render_img(): Pos: [2.64153703 0.         2.88468928] angle 4.836049502872963
22:42:25|gym-duckietown|simulator.py:1435|_render_img(): Pos: [2.6415370304316736, 0, 2.884689280745409] angle -1.4471358043066236
22:42:25|gym-duckietown|simulator.py:1435|_rend

22:42:27|gym-duckietown|simulator.py:1435|_render_img(): Pos: [2.636784715558497, 0, 2.9466429293091116] angle -1.8409273286216579
22:42:27|gym-duckietown|simulator.py:1435|_render_img(): Pos: [2.6365981286107925, 0, 2.9472974261127955] angle -1.8561000916080839
22:42:27|gym-duckietown|simulator.py:1435|_render_img(): Pos: [2.6363677055890418, 0, 2.948058183798971] angle -1.8736977560446664
22:42:27|gym-duckietown|simulator.py:1435|_render_img(): Pos: [2.6360212847312807, 0, 2.9491379045998087] angle -1.888829797322253
22:42:27|gym-duckietown|simulator.py:1435|_render_img(): Pos: [2.635639579254406, 0, 2.950274660287337] angle -1.9006736644769897
22:42:27|gym-duckietown|simulator.py:1435|_render_img(): Pos: [2.6350868405822165, 0, 2.9518664475631398] angle -1.9093540491939542
22:42:27|gym-duckietown|simulator.py:1435|_render_img(): Pos: [2.634453264876578, 0, 2.9536428347483206] angle -1.917439324731425
22:42:28|gym-duckietown|simulator.py:1435|_render_img(): Pos: [2.633745912820156, 0

...

22:46:27|gym-duckietown|simulator.py:1435|_render_img(): Pos: [2.15781388728745, 0, 3.3396347080759115] angle -0.8369209346239602
22:46:27|gym-duckietown|simulator.py:1435|_render_img(): Pos: [2.15830053497032, 0, 3.3401863710833575] angle -0.8589449949511926
22:46:27|gym-duckietown|simulator.py:1435|_render_img(): Pos: [2.1596925336078816, 0, 3.341810238975955] angle -0.8653156879741635
22:46:27|gym-duckietown|simulator.py:1435|_render_img(): Pos: [2.1618642751679618, 0, 3.3444008896408706] angle -0.880952823858961
22:46:27|gym-duckietown|simulator.py:1435|_render_img(): Pos: [2.16237289926583, 0, 3.345027517840781] angle -0.8969901256086981
22:46:27|gym-duckietown|simulator.py:1435|_render_img(): Pos: [2.16315831219909, 0, 3.3460535524967394] angle -0.9379278635570472
22:46:28|gym-duckietown|simulator.py:1435|_render_img(): Pos: [2.162659821521666, 0, 3.3453559718290955] angle -0.9627505513118413
22:46:28|gym-duckietown|simulator.py:1435|_render_img(): Pos: [2.1622780328782887, 0, 3.

22:46:29|gym-duckietown|simulator.py:1435|_render_img(): Pos: [2.1964545208259696, 0, 3.398292857337418] angle -0.765493283228329
22:46:29|gym-duckietown|simulator.py:1435|_render_img(): Pos: [2.1984468676695594, 0, 3.4002346244297006] angle -0.7795910456176326
22:46:29|gym-duckietown|simulator.py:1435|_render_img(): Pos: [2.2000853590376086, 0, 3.401923094752253] angle -0.8212478245150224
22:46:29|gym-duckietown|simulator.py:1435|_render_img(): Pos: [2.202565502562907, 0, 3.404650598773216] angle -0.8444760922702718
22:46:29|gym-duckietown|simulator.py:1435|_render_img(): Pos: [2.20558209389406, 0, 3.40814533997943] angle -0.8729240737993623
22:46:29|gym-duckietown|simulator.py:1435|_render_img(): Pos: [2.2087850794802653, 0, 3.4120594507681905] angle -0.8970469486410856
22:46:29|gym-duckietown|simulator.py:1435|_render_img(): Pos: [2.210977086858572, 0, 3.4149347655407403] angle -0.9418240583152502
22:46:29|gym-duckietown|simulator.py:1435|_render_img(): Pos: [2.213608940322989, 0, 3

22:46:30|gym-duckietown|simulator.py:1435|_render_img(): Pos: [2.218680350964843, 0, 3.4212616584952116] angle -1.1814310872756104
22:46:30|gym-duckietown|simulator.py:1435|_render_img(): Pos: [2.2198263615702185, 0, 3.4240457850693176] angle -1.1791897718782742
22:46:31|gym-duckietown|simulator.py:1435|_render_img(): Pos: [2.220676315774503, 0, 3.4260512523682713] angle -1.160663867766857
22:46:31|gym-duckietown|simulator.py:1435|_render_img(): Pos: [2.22148629003191, 0, 3.427864893631749] angle -1.1408811059140445
22:46:31|gym-duckietown|simulator.py:1435|_render_img(): Pos: [2.2223837200233807, 0, 3.4297683921672912] angle -1.119593790088252
22:46:31|gym-duckietown|simulator.py:1435|_render_img(): Pos: [2.2234376174752275, 0, 3.4318595116103894] angle -1.0883342910203733
22:46:31|gym-duckietown|simulator.py:1435|_render_img(): Pos: [2.2245799941721844, 0, 3.4339116166749717] angle -1.0373356110928782
22:46:31|gym-duckietown|simulator.py:1435|_render_img(): Pos: [2.22538772365329, 0,

EPISODE DONE IN 21.228818655014038
Total T: 671 Episode Num: 1 Episode T: 671 Reward: -1562.248790


KeyboardInterrupt: 

# Stabilizing DDPG

As you may notice, the above model performs poorly or doesn't converge. Your job is to improve it; first in the notebook, later in the AIDO submission. This last part of the assignment consists of four sections:

**1. There are subtle, but important, bugs that have been introduced into the code above. Your job is to find them, and explain them in your `reinforcement-learning-answers.txt`. You'll want to reread the original [DQN](https://deepmind.com/research/publications/human-level-control-through-deep-reinforcement-learning) and [DDPG](https://arxiv.org/abs/1509.02971) papers in order to better understand the issue, but by answering the following subquestions (*please put the answers to these in the submission for full credit*), you'll be on the right track:**

   a) Read some literature on actor-critic methods, including the original [actor-critic](https://papers.nips.cc/paper/1786-actor-critic-algorithms.pdf) paper. What is an issue that you see related to *non-stationarity*? Define what _non-stationarity_ means in the context of machine learning and how it relates to actor-critic methods. In addition, give some hypotheses on why reinforcement learning is much more difficult (from an optimization perspective) than supervised learning, and how the answer to the previous question and this one are related.
   
   The actor update at the same rate as the critic. This is problematic because the critic does not have "enough" time to evaluate the actor's continuously changing policy. 
   
   Non-stationarity, in the context of machine learning, is the fact that for some input, the output is not always be the same. It's easy to visualize for supervised learning. For some input, the label should always be the same, the network should then always predict something close to that label. If it is not the case, there is an outside influence that is not being encapsulated by the input making the learning much more difficult if not completely impossible.
   
   Non-stationarity, in the context of reinforcement learning, describe a situation where the Markov Decision Process (MDP) changes over time. For example, in our case, the policy continuously change. In turn, this means that the critic cannot correctly evaluate the policy over multiple samples. This will negatively affect the accuracy of the reward function. 
   
   What makes reinforcement learning harder than supervised learning is the fact that problems are more often non-stationary. Tricks have then to used to make the training of a model possible. One of those, when using an actor-critic algorithm, is to reduce the actor learning rate so that beta / gamma -> 0 where beta is the actor's learning rate and gamma is the critic learning rate. The policy that the actor modelizes is then updated at a much slower rate than the reward function modelized by the critic. This act as an approximation of a stationary MDP.

   b) What role does the replay buffer play in off-policy reinforcement learning? It's most important parameter is `max_size` - how does changing this value (answer for both increasing and decreasing trends) qualitatively affect the training of the algorithm?
   
   The replay buffer is a table that stores what a reinforcement model experiences during episodes. One entry in the replay buffer correspond to some state, the action taken based on it, the reward received for taking that action in that state and the next state. The role of the replay buffer is to provide a dataset from which the actor and the critic can learn. By taking random batch of samples from it, the correlation between each observation is reduced which is important to actually enforced the Markov property.
   
   When the replay buffer reach its maximum capacity, a randomly selected record in the buffer is replaced by a new one. Also, since a training batch is uniformly sampled, old observations have as much chance to used for training than recent one. The bigger the buffer, the more records is needed before starting to replace old ones. This means that the model will use older observations in its training for longer. The smaller the replay buffer, the faster old records are replaced by recent ones.
   
   A lot of problems in reinforcement learning do not actually respect the Markov property since the best action for a certain state often depends on previous observations. Intuitively, for a lot a of problem, a bigger time difference between two observations leads to a smaller correlation between the two. Therefore, a smaller replay buffer has a higher probability to break the Markov property than a bigger one. For the same reason, the data in a smaller replay buffer has also a lower probability to be iid. However, it is important for the model to learn from recent experience since the more learning the model has done, the better its predictions should be. Recent experience should also be a better indication of the policy and reward function to follow.
   
   In the end, choosing a replay buffer size is a balancing act between how decorrelated the data should be and how much recent experience should weight in the training.

   c) **Challenge Question:** Briefly, explain how automatic differentiation works. In addition, expand on the difference between a single-element tensor (that `requires_grad`) and a scalar value as it relates to automatic differentiation; when do we want to backpropogate through a single-element tensor, and when do we not? Take a close look at the code and how losses are being backpropogated. On paper or your favorite drawing software, draw out the actor-critic architecture *as described in the code*, and label how the actor and critic losses are backpropogated. On your diagram, highlight the particular loss that will cause issues with the above code, and fix it.
   
   For each tensor, which has its require_grad attribute set to true, a history of all the operations applied on it is compiled. This is used to build a computional graph which represent all the interaction between each tensor used in a operation. Once the function backward is called on a result from some operation, the graph is traversed from the end to the start. At each vertex, a gradient is computed, stored and propagated to the vertex after it, by doing the derivative of the operation. This process is repeated until a vertex corresponding to one of the starting tensor is reached. If one of the intermediate vertex is a scalar, the gradient will not be propagated to the vertex after it. This is an important feature because, sometimes, like in the actor-critic algorithm, some operations are calculated but we do not want to actually compute their gradient. Indeed, we normally compute the gradient to update the weights of the model which use the corresponding operation. For example, when computing the target Q, the critic and the actor is used. However, we only want to update the critic based on the gradient from MSE loss between the current and the target Q and the computation of the current Q, not with the gradient from computing the target Q.
   
For the next section, please pick **either** the theoretical or the practical pathway. If you don't have access to the necessary compute, for the exercise, please do the theoretical portion. 
   
_Theoretical Component_ 

**2. We discussed a case study of DQN in class. The original authors used quite a few tricks to get this to work. Detail some of the following, and explain what problem they solve in training the DQN:**

a) Target Networks

   This consist in periodically cloning, every 2500 timesteps in the paper, the network used to compute the Q-value as a target network. It is used to compute a target Q-value which is use when the Q-value to compute the loss. Using a target network help stabilizing the training. In turn, this help converging faster.

b) Annealed Learning Rates

c) Replay Buffer

   They wait for the replay buffer to be full at 20% before starting training the neural network for predicting the Q-value. It is important to accumulate some data in the replay buffer before starting the training since the batch would not be very random before that. They also add a sample only every 4 timesteps probably to reduce the correlation between each observations added to the replay buffer.
 
d) Random Exploration Period

   Waiting for the replay buffer to reach 20% of its capacity also act as a random exploration period since during that time the model's weights are completely random. Indeed, even if they use a greedy policy, if the reward function is completely random, so is the policy. At each timestep, they aso use a probability epsilon to decide if the next action will sampled at random or come from their greedy policy. this means that even after starting the training of the model, and reducing the randomness of the reward function, some exploration is still impose.

e) Preprocessing the Image

   The stacking of multiple frames as one observation helps generating more meaningful data for the training. Indeed, one action often take more than one frame to be actually completed. Hence, treating every frame as one individual observation could flood the training data with noise.


**3. Read about either [TD3](https://arxiv.org/abs/1802.09477) or [Soft Actor Critic](https://arxiv.org/abs/1801.01290); for your choice, summarize what problems they are addressing with the standard actor-critic formulation, and how they solve them**

In the Soft Actor Critic paper, they sustain the idea that, while algorithm like Deep Deterministic Poilcy Gradient are successfully used to trained model with continuous state and action spaces in a sample-efficient manner, the condition in which they are able to converge are very brittle. This means that the more complex the environment, it becomes exponentially harder to make a model training converge.

To that end, the authors introduce a maximum entropy objective in the standard reinforcement learning objective function. Instead of only maximizing the reward, the objective also tries to maximize the entropy of the possible actions for some state. The balance between maximizing the reward and the entropy is controlled by a single hyperparameter alpha that multiplies the entropy objective. The model can then learn a balance between picking the action that gives the best possible reward and keeping a uniform distribution over the possible actions for some state. The paper present this new objective as a solution to training more robust model since the model is trained to maximize the reward "while acting as randomly as possible." [(Haarnoja, Tuomas et al., 2018)](https://arxiv.org/abs/1801.01290) Instead of simply having an exploration period at the beginning of the training, the model is encouraged to explore for the whole process.


_Practical Component_ 

**2. [Optional - if you have access to compute] Using your analysis from the reward engineering ablation, train two agents (after you've found the bugs in DDPG) - one with the standard, `gym-duckietown` reward, and another with the parameters of your choosing. Report each set of parameters, and describe qualitatively what type of behavior the agent produces.**

If you don't have the resources to actually train these agents, instead describe what types of behaviors each reward function might prioritize.

**3. [Optional - if you have access to compute] Using the instructions [here](http://docs.duckietown.org/DT19/AIDO/out/embodied_rl.html), use the saved policy files from this notebook and submit using the template submission provided through the AIDO submission. Report your best submission number (i.e the one you'd like to be graded) in `reinforcement-learning-answers.txt`**